In [13]:
import os
import sys
import json
from pathlib import Path
import pandas as pd

import argparse


ENT_TYPES = ['Task', 'Method', 'Material', 'Metric', 'OtherScientificTerm', 'Generic']
REL_TYPES = {
    'FEATURE-OF': 'Asym', 
    'PART-OF': 'Asym', 
    'USED-FOR': 'Asym', 
    'EVALUATE-FOR': 'Asym',
    'HYPONYM-OF': 'Asym', 
    'COMPARE': 'Sym',
    'CONJUNCTION': 'Sym',
}
RAWDATAFILES = {
    "train": "training_complete.jsonl",
    "val": "validation_complete.jsonl",
    "test": "testing_with_paper_release.jsonl"
}

def print_progress(curr, full, desc='', bar_size=50):    
    bar = int((curr+1)/full*bar_size)
    sys.stdout.write(f"\r{desc}[{'='*bar}{' '*(bar_size-bar)}] {curr+1}/{full}")
    sys.stdout.flush()
    if curr+1==full: print()
    

def getInputDataset(data_split, section):
    main_path = str((Path().absolute()).parents[0])
    filepath = f"{main_path}/PL-Marker/_scire_models/{section}/{data_split}_re.json"
    with open(filepath, 'r') as json_file:
        json_list = list(json_file)
    return [json.loads(json_str) for json_str in json_list]

def addTripEnt(allEnt, tripEnt, ent):
    try:
        tripEnt[ent] = allEnt[ent]
    except:
        tripEnt[ent] = None
    return tripEnt

def getTripSeq(data, sym=True):
    all_sentences = [j for i in data["sentences"] for j in i]
    flatten_ner = [j for i in data["predicted_ner"] for j in i]
    flatten_re  = [j for i in data["predicted_re"] for j in i[1]]
    tripSeq = ""
    tripEnt = {}
    allEnt = {}

    for ner in flatten_ner:
        ent = " ".join(all_sentences[ner[0]:ner[1]+1])
        allEnt[ent] = ner[2]

    for rel in flatten_re:
        s = " ".join(all_sentences[rel[0][0]:(rel[0][1]+1)])
        o = " ".join(all_sentences[rel[1][0]:(rel[1][1]+1)])
        p = rel[2]
        tripSeq += f"{s} {p} {o}. "
        if sym and REL_TYPES[p]=='Sym':
            tripSeq += f"{o} {p} {s}. "
        tripEnt = addTripEnt(allEnt, tripEnt, s)
        tripEnt = addTripEnt(allEnt, tripEnt, o)
        
    freeEnt = allEnt.copy()
    for key, val in tripEnt.items(): freeEnt.pop(key)

    return tripEnt, freeEnt, tripSeq

def getFreeEntSeq(freeEnt):
    freeEntList = {}
    for k, v in freeEnt.items():
        if v not in list(freeEntList.keys()): 
            freeEntList[v] = [k]
        else:
            freeEntList[v].append(k)
    freeEntSeq = ""
    for entType, ents in freeEntList.items():
        if len(ents)==1:
            freeEntSeq += f"{entType} is {ents[0]}. "
        else:
            freeEntSeq += f"{entType} are {', '.join(ents[:-1])}"
            freeEntSeq += f", and {ents[-1]}. "
    return freeEntSeq

def getInputDF(data_split, section="abstract"):
    dataset = getInputDataset(data_split, section)
    dataset_list = []
    for i, data in enumerate(dataset):
        tripEnt, freeEnt, tripSeq = getTripSeq(data)
        # for entType in ENT_TYPES:
        freeEntSeq = getFreeEntSeq(freeEnt)
        row = {
            "paper_id": data['doc_key'], 
            "input_seq": tripSeq+freeEntSeq
        }
        dataset_list.append(row)
        # if i>0: break
    return pd.DataFrame(dataset_list)


def getTargetDF(data_split):
    main_path = str((Path().absolute()).parents[0])    
    filepath = f"{main_path}/MuP_sum/dataset/{RAWDATAFILES[data_split]}"
    with open(filepath, 'r') as json_file:
        json_list = list(json_file)
    dataset_list = []
    data_len = len(json_list)
    for i, json_str in enumerate(json_list):
        data = json.loads(json_str)
        dataset_list.append({
            "paper_id": data["paper_id"], 
            "target_seq": data["summary"]
        })   
        # print_progress(i, data_len, desc=f'Loading summary ({data_split})')
    return pd.DataFrame(dataset_list)
    
    



In [14]:
data_split = "train"
section="abstract"

In [8]:
input_df = getInputDF(data_split, section)
input_df

,paper_id,input_seq
0,SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,average CVloo stability FEATURE-OF kernel ridg...
1,SP:b80bc890180934092cde037b49d94d6e4e06fad9,episodic memories USED-FOR catastrophic forget...
2,SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,shared learning USED-FOR Multi - Task Learning...
3,SP:a1e2218e6943bf138aeb359e23628676b396ed66,fuel optimization problem USED-FOR hybrid elec...
4,SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,RNA HYPONYM-OF genetic macromolecule. graph - ...
...,...,...
8374,SP:77d59e1e726172184249bdfdd81011617dc9c208,Quantum machine learning methods USED-FOR lear...
8375,SP:e58dc2d21175a62499405b7f4c3a03b135530838,generative models USED-FOR priors. priors USED...
8376,SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,deep learning approaches USED-FOR symbolic reg...
8377,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,Discrete Walsh - Hadamard Transform ( DWHT ) C...


In [15]:
target_df = getTargetDF(data_split)
target_df

,paper_id,target_seq
0,SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,This paper investigates kernel ridge-less regr...
1,SP:b80bc890180934092cde037b49d94d6e4e06fad9,This paper presents a novel way of making full...
2,SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,This paper proposes a new framework that compu...
3,SP:a1e2218e6943bf138aeb359e23628676b396ed66,This work proposes a deep reinforcement learni...
4,SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,This paper proposes 3 deep generative models b...
...,...,...
18929,SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,This paper presents a RNN-RL based method for ...
18930,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,This paper proposes a new pointwise convolutio...
18931,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,This paper presents a new pointwise convolutio...
18932,SP:63ad3be1dae7ede5c02a847304072c1cbc91b1cb,This paper proposes to model various uncertain...


In [16]:
merged_df = pd.merge(input_df, target_df, on='paper_id')
merged_df

,paper_id,input_seq,target_seq
0,SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,average CVloo stability FEATURE-OF kernel ridg...,This paper investigates kernel ridge-less regr...
1,SP:b80bc890180934092cde037b49d94d6e4e06fad9,episodic memories USED-FOR catastrophic forget...,This paper presents a novel way of making full...
2,SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,shared learning USED-FOR Multi - Task Learning...,This paper proposes a new framework that compu...
3,SP:a1e2218e6943bf138aeb359e23628676b396ed66,fuel optimization problem USED-FOR hybrid elec...,This work proposes a deep reinforcement learni...
4,SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,RNA HYPONYM-OF genetic macromolecule. graph - ...,This paper proposes 3 deep generative models b...
...,...,...,...
18929,SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,deep learning approaches USED-FOR symbolic reg...,This paper presents a RNN-RL based method for ...
18930,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,Discrete Walsh - Hadamard Transform ( DWHT ) C...,This paper proposes a new pointwise convolutio...
18931,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,Discrete Walsh - Hadamard Transform ( DWHT ) C...,This paper presents a new pointwise convolutio...
18932,SP:63ad3be1dae7ede5c02a847304072c1cbc91b1cb,graph neural networks ( GNNs ) USED-FOR semi -...,This paper proposes to model various uncertain...


In [37]:
remove_papers = list(merged_df[merged_df['input_seq']==''].index)
remove_papers

[6311, 6312, 6313]

In [39]:
merged_df_drop = merged_df.drop(index=remove_papers)
merged_df_drop

,paper_id,input_seq,target_seq
0,SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,average CVloo stability FEATURE-OF kernel ridg...,This paper investigates kernel ridge-less regr...
1,SP:b80bc890180934092cde037b49d94d6e4e06fad9,episodic memories USED-FOR catastrophic forget...,This paper presents a novel way of making full...
2,SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,shared learning USED-FOR Multi - Task Learning...,This paper proposes a new framework that compu...
3,SP:a1e2218e6943bf138aeb359e23628676b396ed66,fuel optimization problem USED-FOR hybrid elec...,This work proposes a deep reinforcement learni...
4,SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,RNA HYPONYM-OF genetic macromolecule. graph - ...,This paper proposes 3 deep generative models b...
...,...,...,...
18929,SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,deep learning approaches USED-FOR symbolic reg...,This paper presents a RNN-RL based method for ...
18930,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,Discrete Walsh - Hadamard Transform ( DWHT ) C...,This paper proposes a new pointwise convolutio...
18931,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,Discrete Walsh - Hadamard Transform ( DWHT ) C...,This paper presents a new pointwise convolutio...
18932,SP:63ad3be1dae7ede5c02a847304072c1cbc91b1cb,graph neural networks ( GNNs ) USED-FOR semi -...,This paper proposes to model various uncertain...


In [47]:
issue_doc = pd.read_csv("issue_data.csv")
remove_index = []
for paper_id in list(issue_doc['paper_id']):
    remove_index += list(merged_df[merged_df['paper_id']==paper_id].index)
remove_index
merged_df_drop = merged_df.drop(index=remove_index)
merged_df_drop

,paper_id,input_seq,target_seq
0,SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,average CVloo stability FEATURE-OF kernel ridg...,This paper investigates kernel ridge-less regr...
1,SP:b80bc890180934092cde037b49d94d6e4e06fad9,episodic memories USED-FOR catastrophic forget...,This paper presents a novel way of making full...
2,SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,shared learning USED-FOR Multi - Task Learning...,This paper proposes a new framework that compu...
3,SP:a1e2218e6943bf138aeb359e23628676b396ed66,fuel optimization problem USED-FOR hybrid elec...,This work proposes a deep reinforcement learni...
4,SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,RNA HYPONYM-OF genetic macromolecule. graph - ...,This paper proposes 3 deep generative models b...
...,...,...,...
18929,SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,deep learning approaches USED-FOR symbolic reg...,This paper presents a RNN-RL based method for ...
18930,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,Discrete Walsh - Hadamard Transform ( DWHT ) C...,This paper proposes a new pointwise convolutio...
18931,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,Discrete Walsh - Hadamard Transform ( DWHT ) C...,This paper presents a new pointwise convolutio...
18932,SP:63ad3be1dae7ede5c02a847304072c1cbc91b1cb,graph neural networks ( GNNs ) USED-FOR semi -...,This paper proposes to model various uncertain...


In [44]:
print()

Int64Index([6311, 6312, 6313], dtype='int64')


In [27]:
def count_num_paper(df):
    num_paper = df.groupby(['input_seq']).count()
    num_paper['num_paper'] = 1
    return num_paper.groupby(['paper_id']).count()
count_num_paper(merged_df)

,target_seq,num_paper
paper_id,,
1,2275,2275
2,2844,2844
3,2117,2117
4,1113,1113
5,10,10
6,15,15
7,4,4


In [22]:
count_num_paper(input_df)


,num_paper
input_seq,
1,8379


In [ ]:
input_df = getInputDF(data_split, section)
print("="*40, "INPUT", "="*40)
print(input_df.head())
print(len(input_df))

target_df = getTargetDF(data_split)
print("="*40, "TARGET", "="*40)
print(target_df.head())
print(len(target_df))

merged_df = pd.merge(input_df, target_df, on='paper_id')
print("="*40, "MERGE", "="*40)
print(merged_df.head())
print(len(merged_df))
paper_id = list(merged_df["paper_id"])
print(len(paper_id))
unique_list = list(set(paper_id))
print(len(unique_list))
input_seq_list = list(merged_df["input_seq"])
unique_list = list(set(input_seq_list))
print(len(unique_list))


issue_doc = pd.read_csv("issue_data.csv")
print(issue_doc)

In [49]:
from transformers import BartTokenizer

In [51]:
import os
#comment this if you are not using puffer?
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

Downloading: 100%|██████████| 899k/899k [00:01<00:00, 497kB/s]  
Downloading: 100%|██████████| 456k/456k [00:01<00:00, 364kB/s]  
Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 34.7kB/s]


In [52]:
input_text = "This is an example input sequence."
encoded_input = tokenizer.encode(input_text, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
encoded_input

tensor([[    0,   152,    16,    41,  1246,  8135, 13931,     4,     2]])

In [ ]:
print(type(input_text))

In [3]:
import os
import sys
import json
from pathlib import Path
import pandas as pd
import argparse
import timeit
import logging

ENT_TYPES = ['Task', 'Method', 'Material', 'Metric', 'OtherScientificTerm', 'Generic']
REL_TYPES = {
    'FEATURE-OF': 'Asym', 
    'PART-OF': 'Asym', 
    'USED-FOR': 'Asym', 
    'EVALUATE-FOR': 'Asym',
    'HYPONYM-OF': 'Asym', 
    'COMPARE': 'Sym',
    'CONJUNCTION': 'Sym',
}
RAWDATAFILES = {
    "train": "training_complete.jsonl",
    "val": "validation_complete.jsonl",
    "test": "testing_with_paper_release.jsonl"
}
# Define hyperparameters
MAX_INPUT_LENGTH = 512
MAX_OUPUT_LENGTH = 512
TRIAN_BATCH_SIZE = 2
EVAL_BATCH_SIZE  = 16
NUM_EPOCH  = 5
LEARNING_RATE = 1e-5


def print_progress(curr, full, desc='', bar_size=30):    
    bar = int((curr+1)/full*bar_size)
    sys.stdout.write(f"\r{desc}[{'='*bar}{' '*(bar_size-bar)}] {curr+1}/{full}")
    sys.stdout.flush()
    if curr+1==full: print()
    

def getInputDataset(data_split, section):
    main_path = str((Path().absolute()).parents[0])
    filepath = f"{main_path}/PL-Marker/_scire_models/{section}/{data_split}_re.json"
    print(f"Loading data from: {filepath}")
    with open(filepath, 'r') as json_file:
        json_list = list(json_file)
    return [json.loads(json_str) for json_str in json_list]

def addTripEnt(allEnt, tripEnt, ent):
    try:
        tripEnt[ent] = allEnt[ent]
    except:
        tripEnt[ent] = None
    return tripEnt

def getTripSeq(data, sym=True):
    all_sentences = [j for i in data["sentences"] for j in i]
    flatten_ner = [j for i in data["predicted_ner"] for j in i]
    flatten_re  = [j for i in data["predicted_re"] for j in i[1]]
    tripSeq = ""
    tripEnt = {}
    allEnt = {}

    for ner in flatten_ner:
        ent = " ".join(all_sentences[ner[0]:ner[1]+1])
        allEnt[ent] = ner[2]

    for rel in flatten_re:
        s = " ".join(all_sentences[rel[0][0]:(rel[0][1]+1)])
        o = " ".join(all_sentences[rel[1][0]:(rel[1][1]+1)])
        p = rel[2]
        tripSeq += f"{s} {p} {o}. "
        if sym and REL_TYPES[p]=='Sym':
            tripSeq += f"{o} {p} {s}. "
        tripEnt = addTripEnt(allEnt, tripEnt, s)
        tripEnt = addTripEnt(allEnt, tripEnt, o)
        
    freeEnt = allEnt.copy()
    for key, val in tripEnt.items(): freeEnt.pop(key)

    return tripEnt, freeEnt, tripSeq

def getFreeEntSeq(freeEnt):
    freeEntList = {}
    for k, v in freeEnt.items():
        if v not in list(freeEntList.keys()): 
            freeEntList[v] = [k]
        else:
            freeEntList[v].append(k)
    freeEntSeq = ""
    for entType, ents in freeEntList.items():
        if len(ents)==1:
            freeEntSeq += f"{entType} is {ents[0]}. "
        else:
            freeEntSeq += f"{entType} are {', '.join(ents[:-1])}"
            freeEntSeq += f", and {ents[-1]}. "
    return freeEntSeq

def getInputDF(data_split, section, prototype):
    dataset = getInputDataset(data_split, section)
    dataset_list = []
    for i, data in enumerate(dataset):
        tripEnt, freeEnt, tripSeq = getTripSeq(data)
        # for entType in ENT_TYPES:
        freeEntSeq = getFreeEntSeq(freeEnt)
        row = {
            "paper_id": data['doc_key'], 
            "input_seq": tripSeq+freeEntSeq
        }
        dataset_list.append(row)
        if isinstance(prototype, int):
            if i>=prototype-1: break
    return pd.DataFrame(dataset_list)


def getTargetDF(data_split, prototype):
    main_path = str((Path().absolute()).parents[0])    
    filepath = f"{main_path}/MuP_sum/dataset/{RAWDATAFILES[data_split]}"
    with open(filepath, 'r') as json_file:
        json_list = list(json_file)
    dataset_list = []
    data_len = len(json_list)
    for i, json_str in enumerate(json_list):
        data = json.loads(json_str)
        dataset_list.append({
            "paper_id": data["paper_id"], 
            "target_seq": data["summary"]
        })
        print_progress(i, data_len, desc=f'Loading summary ({data_split})')
        if isinstance(prototype, int):
            if i>=prototype-1: break
    return pd.DataFrame(dataset_list)
    
def removeIssue(df):
    issue_doc = pd.read_csv("issue_data.csv")
    remove_index = []
    for paper_id in list(issue_doc['paper_id']):
        remove_index += list(df[df['paper_id']==paper_id].index)
    return df.drop(index=remove_index)

def prepro_KGData(data_split, section, prototype):
    input_df = getInputDF(data_split, section, prototype)
    target_df = getTargetDF(data_split, prototype)
    merged_df = pd.merge(input_df, target_df, on='paper_id')
    return input_df, target_df
    return removeIssue(merged_df)

In [4]:
input_df, target_df = prepro_KGData("train", section="summary", prototype=None) 

Loading data from: /home/nopphawann/My_Thesis_Playground/PL-Marker/_scire_models/summary/train_re.json
Loading summary (train)[====                          ] 2999/18934

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Loading summary (train)[========================      ] 15412/18934

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
input_df

,paper_id,input_seq
0,SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,stability arguments USED-FOR risk bounds. stab...
1,SP:b80bc890180934092cde037b49d94d6e4e06fad9,compact episodic memory USED-FOR catastrophic ...
2,SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,task - specific representations USED-FOR model...
3,SP:a1e2218e6943bf138aeb359e23628676b396ed66,deep reinforcement learning - based optimizati...
4,SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,deep generative models USED-FOR generation of ...
...,...,...
18929,SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,RNN - RL based method USED-FOR symbolic regres...
18930,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,DHWT USED-FOR transforming. DCT CONJUNCTION DH...
18931,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,DWHT CONJUNCTION DCT. DCT CONJUNCTION DWHT. DC...
18932,SP:63ad3be1dae7ede5c02a847304072c1cbc91b1cb,uncertainty measures PART-OF Graph Convolution...


In [10]:
# input_df.set_index('paper_id', inplace=True)
target_df.set_index('paper_id', inplace=True)
merged_df = pd.concat([input_df, target_df], axis=1)
merged_df


,input_seq,target_seq
paper_id,,
SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,stability arguments USED-FOR risk bounds. stab...,This paper investigates kernel ridge-less regr...
SP:b80bc890180934092cde037b49d94d6e4e06fad9,compact episodic memory USED-FOR catastrophic ...,This paper presents a novel way of making full...
SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,task - specific representations USED-FOR model...,This paper proposes a new framework that compu...
SP:a1e2218e6943bf138aeb359e23628676b396ed66,deep reinforcement learning - based optimizati...,This work proposes a deep reinforcement learni...
SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,deep generative models USED-FOR generation of ...,This paper proposes 3 deep generative models b...
...,...,...
SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,RNN - RL based method USED-FOR symbolic regres...,This paper presents a RNN-RL based method for ...
SP:4706017e6f8b958c7d0825fed98b285ea2994b59,DHWT USED-FOR transforming. DCT CONJUNCTION DH...,This paper proposes a new pointwise convolutio...
SP:4706017e6f8b958c7d0825fed98b285ea2994b59,DWHT CONJUNCTION DCT. DCT CONJUNCTION DWHT. DC...,This paper presents a new pointwise convolutio...


In [11]:
merged_df.reset_index()

,paper_id,input_seq,target_seq
0,SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,stability arguments USED-FOR risk bounds. stab...,This paper investigates kernel ridge-less regr...
1,SP:b80bc890180934092cde037b49d94d6e4e06fad9,compact episodic memory USED-FOR catastrophic ...,This paper presents a novel way of making full...
2,SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,task - specific representations USED-FOR model...,This paper proposes a new framework that compu...
3,SP:a1e2218e6943bf138aeb359e23628676b396ed66,deep reinforcement learning - based optimizati...,This work proposes a deep reinforcement learni...
4,SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,deep generative models USED-FOR generation of ...,This paper proposes 3 deep generative models b...
...,...,...,...
18929,SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,RNN - RL based method USED-FOR symbolic regres...,This paper presents a RNN-RL based method for ...
18930,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,DHWT USED-FOR transforming. DCT CONJUNCTION DH...,This paper proposes a new pointwise convolutio...
18931,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,DWHT CONJUNCTION DCT. DCT CONJUNCTION DWHT. DC...,This paper presents a new pointwise convolutio...
18932,SP:63ad3be1dae7ede5c02a847304072c1cbc91b1cb,uncertainty measures PART-OF Graph Convolution...,This paper proposes to model various uncertain...


In [6]:
target_df

,paper_id,target_seq
0,SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,This paper investigates kernel ridge-less regr...
1,SP:b80bc890180934092cde037b49d94d6e4e06fad9,This paper presents a novel way of making full...
2,SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,This paper proposes a new framework that compu...
3,SP:a1e2218e6943bf138aeb359e23628676b396ed66,This work proposes a deep reinforcement learni...
4,SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,This paper proposes 3 deep generative models b...
...,...,...
18929,SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,This paper presents a RNN-RL based method for ...
18930,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,This paper proposes a new pointwise convolutio...
18931,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,This paper presents a new pointwise convolutio...
18932,SP:63ad3be1dae7ede5c02a847304072c1cbc91b1cb,This paper proposes to model various uncertain...
